In [5]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import math
import re
from datetime import datetime
from datetime import timedelta

In [6]:
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_rows',500)

In [7]:
game = game = pd.read_csv("game_summary.csv")

In [4]:
play = pd.read_csv("play_by_play.csv")

KeyboardInterrupt: 

In [ ]:
play = play.drop(['eventnum', 'eventmsgactiontype',
       'wctimestring', 'neutraldescription', 'person1type',
       'player1_team_id', 'player1_team_city',
       'player1_team_nickname', 'player1_team_abbreviation', 'person2type',
       'player2_team_id', 'player2_team_city',
       'player2_team_nickname', 'player2_team_abbreviation', 'person3type',
       'player3_team_id', 'player3_team_city',
       'player3_team_nickname', 'player3_team_abbreviation',
       'video_available_flag'], axis = 1)
play

In [ ]:
game_dict = {'1946' : [], '1947' : [], '1948' : [], '1949' : [], '1950' : [], '1951' : [], '1952' : [], '1953' : [], '1954' : [], '1955' : [], '1956' : [], '1957' : [], '1958' : [], 
             '1959' : [], '1960' : [], '1961' : [], '1962' : [], '1963' : [], '1964' : [], '1965' : [], '1966' : [], '1967' : [], '1968' : [], '1969' : [], '1970' : [], '1971' : [], 
             '1972' : [], '1973' : [], '1974' : [], '1975' : [], '1976' : [], '1977' : [], '1978' : [], '1979' : [], '1980' : [], '1981' : [], '1982' : [], '1983' : [], '1984' : [], 
             '1985' : [], '1986' : [], '1987' : [], '1988' : [], '1989' : [], '1990' : [], '1991' : [], '1992' : [], '1993' : [], '1994' : [], '1995' : [], '1996' : [], '1997' : [], 
             '1998' : [], '1999' : [], '2000' : [], '2001' : [], '2002' : [], '2003' : [], '2004' : [], '2005' : [], '2006' : [], '2007' : [], '2008' : [], '2009' : [], '2010' : [], 
             '2011' : [], '2012' : [], '2013' : [], '2014' : [], '2015' : [], '2016' : [], '2017' : [], '2018' : [], '2019' : [], '2020' : [], '2021' : [], '2022' : [], '2023' : []}
for row in game.iterrows():
    season = str(row[1]["season"])
    game_dict[season].append(row[1]["game_id"])

In [ ]:
def calc_time(start_time, end_time):
    time = re.compile("\d+")
    start_time = time.findall(start_time)
    end_time = time.findall(end_time)
    delta = timedelta(seconds = float(start_time[1]) - float(end_time[1]), minutes = float(start_time[0]) - float(end_time[0]))
    return math.ceil(delta.total_seconds()/60)

In [ ]:
player = pd.read_csv("time_data.csv")
player["rebounds"] = [np.nan] * player.shape[0]
player["ORB"] = [0] * player.shape[0]
player["DRB"] = [0] * player.shape[0]
player["MP"] = [0] * player.shape[0]
player["points"] = [0] * player.shape[0]
player["three_pointers"] = [0] * player.shape[0]
player["start_time"] = [np.nan] * player.shape[0]
player

In [ ]:
count = 0
margin = 0
period = 1
eventmsgtype = 1
period_index = 2
timestring = 3
homedescription = 4
visitordescription = 5
scoremargin = 7
player1_id = 8
player1_name = 9
player2_id = 10
player2_name = 11
player3_id = 12
player3_name = 13
pattern = re.compile("\d+")
print("starting...")
for key, values in game_dict.items():
    key = int(key)
    if key < 1996:
        continue
    print(key, end = ", ")
    for game in values:
        margin = 0
        for row in play.loc[play["game_id"] == game].values:
            if (pd.notna(row[player1_name])) & True in (player.loc[(player["name"] == row[player1_name]) & (player["season"] == key), "start_time"]).isna().values:
                player.loc[(player["name"] == row[player1_name]) & (player["season"] == key), "start_time"] = row[timestring]
            
            if (pd.notna(row[player2_name])) & True in (player.loc[(player["name"] == row[player1_name]) & (player["season"] == key), "start_time"]).isna().values:
                player.loc[(player["name"] == row[player2_name]) & (player["season"] == key), "start_time"] = row[timestring]  
                
            if (pd.notna(row[player3_name])) & True in (player.loc[(player["name"] == row[player1_name]) & (player["season"] == key), "start_time"]).isna().values:
                player.loc[(player["name"] == row[player3_name]) & (player["season"] == key), "start_time"] = row[timestring]
                
            if row[eventmsgtype] == 1:
                try:
                    if row[scoremargin] == "TIE":
                        points = abs(0 - margin)
                    else:
                        points = abs(int(row[scoremargin]) - margin)
                    player.loc[(player["name"] == row[player1_name]) & (player["season"] == key), "points"] += points
                    if points == 3:
                        player.loc[(player["name"] == row[player1_name]) & (player["season"] == key), "three_pointers"] += 1
                    if row[scoremargin] == "TIE":
                        margin = 0
                    else:
                        margin = int(row[scoremargin])
                except:
                    pass
            elif row[eventmsgtype] == 4:
                if pd.notna(row[homedescription]):
                    player.loc[(player["person_id"] == row[player1_id]) & (player["season"] == key), "rebounds"] = row[homedescription]
                else:
                    player.loc[(player["person_id"] == row[player1_id]) & (player["season"] == key), "rebounds"] = row[visitordescription]
        
            elif row[eventmsgtype] == 8:
                try:
                    MP = calc_time(player.loc[(player["person_id"] == row[player1_id]) & (player["season"] == key), "start_time"].values[0], row[timestring])
                    player.loc[(player["person_id"] == row[player1_id]) & (player["season"] == key), "MP"] += MP
                    player.loc[(player["person_id"] == row[player1_id]) & (player["season"] == key), "start_time"] = np.nan
                except:
                    pass
            
            elif row[eventmsgtype] == 13:
                for i in player["start_time"].values:
                    if type(i) == str:
                        MP = calc_time(i, row[timestring])
                        player.loc[player["start_time"] != np.nan, "MP"] += MP
                        player.loc[player["start_time"] != np.nan, "start_time"] = np.nan
                period = row[period_index]
            if count % 100000 == 0:
                print(count, end = ", ")
            count += 1
        margin = 0
        for values in player.loc[player["rebounds"].notna()].values:
            try:
                temp = pattern.findall(values[16])
                player.loc[(player["name"] == values[3]) & (player["season"] == key), "ORB"] += int(temp[0])
                player.loc[(player["name"] == values[3]) & (player["season"] == key), "DRB"] += int(temp[1])
            except:
                pass
            
print("Finished")

In [ ]:
player.head(50)

In [ ]:
from pathlib import Path  
filepath = Path('Lumiere_Reasearch/time_data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
player.to_csv(filepath)  